In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

In [2]:
def split_sequences(first_seq, secend_seq, sw_width):
    '''
    該函數將序列數據分割成樣本
    '''
    input_seq1 = np.array(first_seq).reshape(len(first_seq), 1)
    input_seq2 = np.array(secend_seq).reshape(len(secend_seq), 1)
    out_seq = np.array([first_seq[i]+secend_seq[i] for i in range(len(first_seq))])
    out_seq = out_seq.reshape(len(out_seq), 1)
    
    dataset = np.hstack((input_seq1, input_seq2, out_seq))
    print('dataset:\n',dataset)
    
    X, y = [], []
    
    for i in range(len(dataset)):
        # 切片索引從0開始，區間爲前閉後開，所以不用減去1
        end_element_index = i + sw_width
        # 同樣的道理，這裏考慮最後一個樣本正好取到最後一行的數據，那麼索引不能減1，如果減去1的話最後一個樣本就取不到了。
        if end_element_index > len(dataset):
            break
        
        # 該語句實現步長爲1的滑動窗口截取數據功能；
        # 以下切片中，:-1 表示取除最後一列的其他列數據；-1表示取最後一列的數據
        seq_x, seq_y = dataset[i:end_element_index, :-1], dataset[end_element_index-1, -1]
        
        X.append(seq_x)
        y.append(seq_y)
        
        process_X, process_y = np.array(X), np.array(y)
    
    n_features = process_X.shape[2]
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    print('n_features:',n_features)
    return process_X, process_y, n_features

In [3]:
def oned_cnn_model(sw_width, n_features, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))
    
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    model.add(Flatten())
    
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

In [4]:

train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
train_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
sw_width = 3

epoch_num = 1000
verbose_set = 0

train_X, train_y, n_features = split_sequences(train_seq1, train_seq2, sw_width)

# 預測
x_input = np.array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, sw_width, n_features))

model, history = oned_cnn_model(sw_width, n_features, train_X, train_y, x_input, epoch_num, verbose_set)

print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

dataset:
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
train_X:
[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]

 [[60 65]
  [70 75]
  [80 85]]

 [[70 75]
  [80 85]
  [90 95]]]
train_y:
[ 65  85 105 125 145 165 185]

train_X.shape:(7, 3, 2),trian_y.shape:(7,)

n_features: 2
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2, 64)             320       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 64)  